# Convolution Neural Network

# Libraries

In [ ]:
import numpy as np
import tensorflow as tf
import pandas as pd
from tensorflow.python import keras
from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.layers import Dense, Flatten, Conv2D, Dropout, MaxPooling2D

In [ ]:
# Input data files are available in the "../input/" directory.
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Input data

In [ ]:
train=pd.read_csv("/kaggle/input/digit-recognizer/train.csv")
print(train.shape)
train.head()

In [ ]:
test=pd.read_csv("/kaggle/input/digit-recognizer/test.csv")
print(test.shape)
test.head()

In [ ]:
X_train=(train.iloc[:,1:].values).astype('float32')#all pixel values
Y_train=train.iloc[:,0].values.astype('int32')#only labels i.e. target digits
X_test=test.values.astype('float32')

In [ ]:
X_train

In [ ]:
Y_train

In [ ]:
img_rows,img_cols=28,28
input_shape=(img_rows,img_cols,1)

# Image exploration

In [ ]:
#reshaping images to accomodate all images
X_train=X_train.reshape(X_train.shape[0],img_rows,img_cols,1)
X_train.shape

In [ ]:
#reshaping images to accomodate all images
X_test=X_test.reshape(X_test.shape[0],img_rows,img_cols,1)
X_test.shape

In [ ]:
import matplotlib.pyplot as plt
#plot first image
plt.imshow(X_train[1][:,:,0])
#check image shape
X_train[1].shape

# Image Preprocessing 

In [ ]:
#Normalization
X_train /= 255
X_test /= 255

In [ ]:
#One-Hot Encoding
#Convert class vectors to binary class matrices
from keras.utils import to_categorical
Y_train=to_categorical(Y_train,10)

# Neural Network Structure

In [ ]:
#first convolutional layer followed by max pooling
model=Sequential()
model.add(Conv2D(32, kernel_size=(3, 3),
                  activation='relu',
                  input_shape=input_shape))
model.add(MaxPooling2D(pool_size=(2, 2), strides=None))
print(model.output_shape)

In [ ]:
#later hidden layers
#2 convolutional layers followed by 2 MaxPooling Layers each
#after which, there is a flatten layer, followed by 2 dense layers
model.add(Conv2D(32, kernel_size=(3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2), strides=None))
model.add(Conv2D(32, kernel_size=(3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2), strides=None))
model.add(Flatten())
model.add(Dense(240,activation='relu'))
model.add(Dense(240,activation='relu'))
model.add(Dense(10,activation='softmax'))

print("input shape ",model.input_shape)
print("output shape ",model.output_shape)

# Compile

In [ ]:
from keras import optimizers
model.compile(loss=keras.losses.categorical_crossentropy,
             optimizer='adam', 
             metrics=['accuracy'])

# Train the model

In [ ]:
from keras.preprocessing import image
gen=image.ImageDataGenerator()

In [ ]:
from sklearn.model_selection import train_test_split
X = X_train
Y = Y_train
X_train, X_val, Y_train, y_val = train_test_split(X_train, Y_train, test_size=0.10, random_state=42)

In [ ]:
#train the model
batch_size = 420
epochs = 10
model.fit(X_train, Y_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(X_val, y_val))

# Predict using model

In [ ]:
#predict first image in the test set
model.predict(X_test[:1])

# Data Augumentation

In [ ]:
from keras.preprocessing.image import ImageDataGenerator
datagen = ImageDataGenerator(
        rotation_range=10, zoom_range = 0.05,  
        width_shift_range=0.1, height_shift_range=0.1,
        horizontal_flip=False, vertical_flip=False)

datagen.fit(X_train)

In [ ]:
#train the model
batch_size = 140
epochs = 30
model.fit(X_train, Y_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(X_val, y_val))

# Predict new Model

In [ ]:
#predict first image in the test set
model.predict(X_test[:1])

In [ ]:
predictions = model.predict_classes(X_test, verbose=0)
submissions=pd.DataFrame({"ImageId": list(range(1,len(predictions)+1)),
                         "Label": predictions})
submissions.to_csv("DR.csv", index=False, header=True)
print(submissions)

In [ ]:
submissions.to_csv('csv_to_submit.csv', index = False)